Install MySQL Connector

In [1]:
!pip install mysql-connector-python

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/99/72/9bcbc7d6fe122fcbb4f948fd79c18387e84c4df8762aa55eb230acf1193b/mysql_connector_python-8.1.0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
      --------------------------------------- 10.2/527.0 kB ? eta -:--:--
      --------------------------------------- 10.2/527.0 kB ? eta -:--:--
     -- ---------------------------------- 30.7/527.0 kB 220.2 kB/s eta 0:00:03
     -- ---------------------------------- 41.0/527.0 kB 219.4 kB/s eta 0:00:03
     ----- ------------------------------- 71.7/527.0 kB 328.6 kB/s eta 0:00:02
     ---------- ------------------------- 153.6/527.0 kB 612.6 kB/s eta 0:00:01
     ----------------- ------------------ 256.0/527.0 kB 874.6 kB/s eta 0:00:01
     --------------------- -------------- 317.4/527.0 kB 936.6 kB/s eta 0:00:01
     -------------------------------------  522.2/527.0 kB 1.4 MB/


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Connect to the database using a Python client

In [1]:
import mysql.connector as connector
connection=connector.connect(user="root",password="")
cursor = connection.cursor()

In [2]:
# Show list of available databases
cursor.execute("show databases")
for db in cursor:
    print(db)

('global_super_store',)
('information_schema',)
('little_lemon',)
('mysql',)
('performance_schema',)
('phpmyadmin',)
('test',)


In [3]:
cursor.execute("use little_lemon")

In [19]:
print(connection.database)

little_lemon


In [20]:
# Show list of available tables in Little Lemon DB
cursor.execute("show tables")
for table in cursor:
    print(table)

('bookings',)
('employees',)
('menuitems',)
('menus',)
('orders',)


Create GetMaxQuantity Procedure

In [21]:
cursor.execute("DROP PROCEDURE IF EXISTS GetMaxQuantity;")

max_quantity_query="""
CREATE PROCEDURE GetMaxQuantity()

BEGIN
SELECT MAX(Quantity) as maxQuantity
FROM Orders;
END

"""

cursor.execute(max_quantity_query)

Call GetMaxQuantity procedure and print the result

In [23]:
cursor.callproc("GetMaxQuantity")

results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]
    
print(columns)
for data in dataset:
    print(data)

['maxQuantity']
(2,)


Create ManageBooking procedure

In [33]:
cursor.execute("DROP PROCEDURE IF EXISTS ManageBooking;")

manage_booking_query="""
CREATE PROCEDURE ManageBooking()

BEGIN
SELECT bookings.BookingID,
CONVERT(bookings.BookingSlot, CHAR) as BookingSlot
FROM Bookings
ORDER BY bookings.BookingSlot;
END

"""

cursor.execute(manage_booking_query)

Call ManageBooking procedure and print the result

In [34]:
cursor.callproc("ManageBooking")

results = next(cursor.stored_results())
dataset = results.fetchall()

for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]
    
print(columns)
for data in dataset:
    print(data)

['BookingID', 'BookingSlot']
(3, '15:00:00')
(4, '17:30:00')
(7, '18:00:00')
(9, '18:00:00')
(5, '18:30:00')
(1, '19:00:00')
(2, '19:00:00')
(10, '19:00:00')
(8, '19:00:00')
(11, '19:30:00')
(6, '20:00:00')


Create UpdateBooking Procedure

In [53]:
cursor.execute("DROP PROCEDURE IF EXISTS UpdateBooking;")

update_booking_query="""
CREATE PROCEDURE UpdateBooking(IN existing_id int, IN new_time time)

BEGIN
UPDATE Bookings 
SET bookings.BookingSlot = new_time
WHERE BookingID = existing_id;

SELECT 
    CONCAT('Booking ', existing_id, 'has been updated.');
END

"""

cursor.execute(update_booking_query)

In [9]:
cursor.callproc("UpdateBooking",[3,'15:00:00'])
#cursor.callproc(ProcedureName, Params)

(3, '15:00:00')

In [31]:
cursor.execute("""SELECT BookingID, TableNo, CONVERT(BookingSlot, CHAR) as BookingTime FROM Bookings;""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

('BookingID', 'TableNo', 'BookingTime')
(1, 12, '20:00:00')
(2, 12, '19:00:00')
(3, 19, '15:00:00')
(4, 15, '17:30:00')
(5, 5, '18:30:00')
(6, 8, '20:00:00')
(7, 8, '18:00:00')
(8, 5, '19:00:00')
(9, 8, '18:00:00')
(10, 5, '19:00:00')
(14, 1, '19:00:00')


Create CancelBooking Procedure

In [32]:
cursor.execute("DROP PROCEDURE IF EXISTS CancelBooking;")

cancel_booking_query="""
CREATE PROCEDURE CancelBooking(IN booking_id int)

BEGIN
DECLARE isExist boolean;

SELECT Count(1) 
INTO isExist
FROM Orders 
WHERE BookingID = booking_id;

IF isExist THEN
    SELECT CONCAT('Cannot cancel Booking ', booking_id);
ELSE
    START TRANSACTION;
    DELETE FROM Bookings
    WHERE BookingID = booking_id; 
    COMMIT;
    SELECT CONCAT('Booking ', booking_id, 'has been cancelled.');
    
END IF;

END

"""

cursor.execute(cancel_booking_query)

In [28]:
cursor.callproc("CancelBooking",[13])
#cursor.callproc(ProcedureName, Params)

(13,)

Create AddBooking Procedure

In [33]:
cursor.execute("DROP PROCEDURE IF EXISTS AddBooking;")

add_booking_query="""
CREATE PROCEDURE AddBooking(IN table_no int, IN first_name varchar(45), IN last_name varchar(45), IN booking_slot time)

BEGIN
DECLARE isExist boolean;

SELECT Count(1) 
INTO isExist
FROM Bookings
WHERE TableNo = table_no
AND BookingSlot = booking_slot;

IF isExist THEN
    SELECT CONCAT('Table No ', table_no, ' has been reserved at ', booking_slot, '. Please book another table!');
ELSE
    START TRANSACTION;
    INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot)
    VALUES(table_no, first_name, last_name, booking_slot);
    COMMIT;
    SELECT CONCAT('Succesfully booked table no ', table_no);
END IF;

END

"""

cursor.execute(add_booking_query)

In [30]:
cursor.callproc("AddBooking",[1, 'Fajri', 'Widiadi', '19:00:00'])
#cursor.callproc(ProcedureName, Params)

(1, 'Fajri', 'Widiadi', '19:00:00')